# **Problem 4: Automated video surveillance**

In [1]:
import cv2
import numpy as np

In [3]:
video = cv2.VideoCapture('q4.mp4')

if not video.isOpened():
    print("Error: Failed to open video file!")
    exit()

# Get video properties
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = video .get(cv2.CAP_PROP_FPS)

# Initialize video writer to save output
video_codec = cv2.VideoWriter_fourcc(*'XVID')
output_stream = cv2.VideoWriter('final_output.avi', video_codec, frame_rate, (frame_width, frame_height))

# Create background subtractor for motion detection
bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

prev_frame = None

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break
    
    # Convert to grayscale and Apply Gaussian blur to reduce noise
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply background subtraction to detect movement
    fg_mask = bg_subtractor.apply(blurred)
    
    _, thresh = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)
    
    # Use morphological operations (erosion and dilation)
    morph_kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.erode(thresh, morph_kernel, iterations=1)   # Reduce noise
    thresh = cv2.dilate(thresh, morph_kernel, iterations=3)   # Enhance moving regions
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize bounding box for detected movement
    x_min, y_min, x_max, y_max = frame_width, frame_height, 0, 0
    movement_detected = False  
    
    for contour in contours:
        if cv2.contourArea(contour) > 1500: 
            movement_detected = True
            x, y, w, h = cv2.boundingRect(contour)
            x_min = min(x_min, x)
            y_min = min(y_min, y)
            x_max = max(x_max, x + w)
            y_max = max(y_max, y + h)
    
    # Draw bounding box around detected moving object
    if movement_detected:
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    
    output_stream.write(frame)
    cv2.imshow('Motion Detection', frame)
    
    # Exit loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
output_stream.release()
cv2.destroyAllWindows()